In [14]:
import folium
import json
import pandas as pd
from datetime import datetime
from shapely.geometry import Point
from geopandas import GeoDataFrame
import geopandas as gpd
from folium.plugins import MarkerCluster
import plotly.express as px
#conda install -c plotly plotly-orca
#plotly.io.orca.config.executable = '/path/to/orca'
#plotly.io.orca.config.save()

# Creating Map

This is only needed for foluim Choropleth. For Plotly, a base map can be added by mapbox_style input parameter. 

In [ ]:
m_chrono = folium.Map(location=[55.9533, 3.1883], zoom_start=5)

# key = #Ordnance Survey (OS) API_KEY 
# layer = 'Light_3857'
# zxy_path = 'https://api.os.uk/maps/raster/v1/zxy/{}/{{z}}/{{x}}/{{y}}.png?key={}'.format(layer, key)

# # Create a new Folium map
# # Ordnance Survey basemap using the OS Data Hub OS Maps API centred on the boundary centroid location
# # Zoom levels 7 - 16 correspond to the open data zoom scales only
# m_chrono = folium.Map(location=[55.9533, -3.1883],
#                min_zoom=7,
#                max_zoom=16,
#                tiles=zxy_path,
#                attr='Contains OS data © Crown copyright and database right {}'.format(datetime.year),
#                clustered_marker = False)

# Read Data

In [15]:
shp_data = gpd.read_file(location +"GBR_adm2.shp") # GBR ADM2 Dataset Should be Downloaded
shp_data= shp_data[shp_data.NAME_1 == 'Scotland'].reset_index(drop=True)
scotland_population=pd.read_excel("Scotland_Population.xlsx") 

In [16]:
#Create geometry column and convert the df to geo df
scotland_stations = pd.read_csv('scotland_station_df.csv')
geometry = [Point(xy) for xy in zip(scotland_stations.longitude, scotland_stations.latitude)]
scotland_stations_geo = GeoDataFrame(scotland_stations, crs="EPSG:4326", geometry=geometry)
#For plotly Geojson version of df is created and saved as Geojson. After creating once, just reading is enough geojson=scot_states
with open("Geojson.json") as f:
      scot_states = json.load(f)

In [17]:
len(scotland_stations_geo)

1407

# Data Preparation 

In [18]:
# Display Unity Area polygons
shp_data_mark = MarkerCluster(name="unity area")
for _, r in shp_data.iterrows():
    #without simplifying the representation of each borough, the map might not be displayed
    #sim_geo = gpd.GeoSeries(r['geometry'])
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'},
                          popup="mahir",)
    geo_j.add_to(shp_data_mark)
shp_data_mark.add_to(m_chrono)

In [19]:
number_of_stations={}
stations = {}
rapid_stations = {}
fast_stations = {}
slow_stations = {}
for j,k in shp_data.iterrows():
    num = 0
    rapid_charger = 0
    fast_charger = 0
    slow_charger = 0
    index_list=[]
    for i,v in scotland_stations_geo.iterrows(): 
        if k.geometry.contains(v.geometry) == True: # check whether a station is in a particular polygon or not
            if v.Charger_Category == "Rapid":
                rapid_charger += 1
            elif v.Charger_Category == "Fast":
                fast_charger += 1
            elif v.Charger_Category == "Slow":
                slow_charger += 1
            index_list.append(i)
            num = num + 1 
    stations[k['NAME_2']] = index_list
    number_of_stations[k['NAME_2']] = num
    rapid_stations[k['NAME_2']] = rapid_charger
    fast_stations[k['NAME_2']] = fast_charger 
    slow_stations[k['NAME_2']] = slow_charger 
chro_pd = pd.DataFrame.from_dict(number_of_stations, orient='index').reset_index(drop=False)
chro_pd = chro_pd.rename(columns = {"index":"NAME_2", 0: "Number_of_Station"})

df_11 = pd.DataFrame.from_dict(rapid_stations, orient='index').reset_index(drop=False)
df_12 = pd.DataFrame.from_dict(fast_stations, orient='index').reset_index(drop=False)
df_13 = pd.DataFrame.from_dict(slow_stations, orient='index').reset_index(drop=False)
df_11["fast"]= df_12[0]
df_11["slow"] = df_13[0]
df_11["Number_of_Station"]= chro_pd["Number_of_Station"]

temp_df_2 = df_11.rename(columns = {"index":"NAME_2", 0: "rapid"})

final_df = shp_data.merge(chro_pd, on = "NAME_2")
final_df_all = shp_data.merge(temp_df_2, on = "NAME_2")

In [20]:
temp_df_2 =temp_df_2.sort_values(by=["Number_of_Station"],ascending=False).reset_index(drop=True)
temp_df_2 ["Population"] = scotland_population["Population"]
temp_df_2 ["Density"]= temp_df_2 .Number_of_Station/(temp_df_2.Population/100000)
final_df_density = shp_data.merge(temp_df_2, on = "NAME_2")

In [21]:
final_df.Number_of_Station.sum()

865

# Create Maps

In [ ]:
# Number of CS per unity area 
folium.Choropleth(
geo_data=final_df,
data=final_df,
columns=['NAME_2',"Number_of_Station"],
key_on="feature.properties.NAME_2",
fill_color='YlGnBu',
fill_opacity=1,
line_opacity=0.2,
legend_name="Number of Stations",
smooth_factor=0,
Highlight= True,
line_color = "#0000",
name = "Number of Stations",
show=False,
overlay=True,
nan_fill_color = "Number of Stations"
).add_to(m_chrono)
m_chrono

In [ ]:
#m_chrono.save('Last_Chrono.html')

In [ ]:
# Number of CS per unity area by Plotly Express
fig_den = px.choropleth_mapbox(final_df, geojson=scot_states, color="Number_of_Station",
                           locations="NAME_2",featureidkey="properties.NAME_2",
                           center={"lat": 55.9533, "lon":-3.1883},
                            color_continuous_scale= 'YlGnBu',#"Viridis"
                            labels={'NAME_2':'Unitary Distinct',"Number_of_Station":"Number of Stations" },
                           mapbox_style="open-street-map", zoom=5)
fig_den.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_den.show() 

In [ ]:
# fig_den.write_html("C:\\Users\\~~.html",width = 600, height = 400))

In [ ]:
# Number of rapid CS per unity area by Plotly Express
fig_den = px.choropleth_mapbox(final_df_all, geojson=scot_states, color="rapid",
                           locations="NAME_2",featureidkey="properties.NAME_2",
                           center={"lat": 55.9533, "lon":-3.1883},
                            color_continuous_scale= 'YlGnBu',#"Viridis"
                            labels={'NAME_2':'Unitary Distinct',"rapid":"Number of Rapid Stations",
                                    "Number_of_Station": "Total Number of Stations"},
                           mapbox_style="open-street-map", zoom=5)
fig_den.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_den.show() 

In [ ]:
# fig_den.write_html("C:\\Users\\~~.html",width = 600, height = 400))

In [ ]:
# Number of CS per 100,000 people per unity area by Plotly Express
fig_den = px.choropleth_mapbox(final_df_density, geojson=scot_states, color="Density",
                           locations="NAME_2",featureidkey="properties.NAME_2",
                           center={"lat": 55.9533, "lon":-3.1883},
                            color_continuous_scale= 'YlGnBu',#"Viridis"
                            labels={'NAME_2':'Unitary Distinct',"Density":"Number of CSs per 100,000 People",
                                    "Number_of_Station": "Total Number of Stations"},
                           mapbox_style="open-street-map", zoom=5)
fig_den.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_den.show() 

In [ ]:
# fig_den.write_html("C:\\Users\\~~.html",width = 600, height = 400))